In [ ]:
## Libraries
import glob
from multiprocessing import cpu_count
import os
import random
import sys

## 3rd party
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
import torch
from torch.utils.data import DataLoader

_path = ".."
if _path not in sys.path:
    sys.path.append(_path)
from lib.dataset import TextArtDataLoader, AlignCollate, ImageBatchSampler
from lib.config import Config
# from lib.preprocess import (pad_image, crop_edges_lr, )

%reload_ext autoreload
%autoreload 2

In [ ]:
DATA_DIR = 'united'

In [ ]:
BATCH_SIZE = 2
# N_WORKERS = cpu_count() - 1
N_WORKERS = 0
CONFIG = Config()

In [ ]:
train_dataset = TextArtDataLoader(DATA_DIR, CONFIG, mode='train')
val_dataset = TextArtDataLoader(DATA_DIR, CONFIG, mode='val')
test_dataset = TextArtDataLoader(DATA_DIR, CONFIG, mode='test')

train_align_collate = AlignCollate(CONFIG, 'train')
val_align_collate = AlignCollate(CONFIG, 'val')

train_batch_sampler = ImageBatchSampler(DATA_DIR, CONFIG, mode='train')
val_batch_sampler = ImageBatchSampler(DATA_DIR, CONFIG, mode='val')

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=N_WORKERS,
                          pin_memory=True,
                          collate_fn=train_align_collate,
                          sampler=train_batch_sampler,
                         )
val_loader = DataLoader(val_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=N_WORKERS,
                          pin_memory=True,
                          collate_fn=val_align_collate,
                          sampler=val_batch_sampler
                         )
test_loader = DataLoader(test_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=N_WORKERS,
                          pin_memory=True,
                          collate_fn=None,
                         )

In [ ]:
for i, (image, wv_tensor, fake_wv_tensor) in enumerate(train_loader):
    print("IMAGE:", image.shape)
    print("WV:", wv_tensor.shape)
    print("Fake WV:", fake_wv_tensor.shape)
    
    print("WVs")
    for wvs in wv_tensor:
        for wv in wvs:
            wv = np.array(wv)
            word, prob = train_loader.dataset.word2vec_model.wv.similar_by_vector(wv)[0]
            print("{}/{:.3f}".format(word, prob), end=' ')
        print()
    
    print("\nFake WVs")
    for fake_wvs in fake_wv_tensor:
        for fake_wv in fake_wvs:
            fake_wv = np.array(fake_wv)
            fake_word, prob = train_loader.dataset.word2vec_model.wv.similar_by_vector(fake_wv)[0]
            print("{}/{:.3f}".format(fake_word, prob), end=' ')
        print()
    
    images = np.array(image)
    for img in images:
        img = img.transpose(1, 2, 0)
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.show()
    break

In [ ]:
mod = train_dataset.word2vec_model

In [ ]:
mod.wv.similar_by_word('cat', topn=None).shape

In [ ]:
mod.wv.similar_by_vector(np.array(word_vector), topn=1)

In [ ]:
mod.wv.similar_by_vector(np.array(word_vector), topn=1000000)[-10:]

In [ ]:
mod.wv.similar_by_vector(vector, topn=20)

In [ ]:
vector1 = mod.wv['cat']
vector2 = mod.wv['dog']
a = np.stack((vector1, vector2))

In [ ]:
random.choice(a)

In [ ]:
similar_words = np.array(mod.wv.similar_by_vector(vector, topn=20))
np.random.choice(similar_words[:,0], 3, replace=False)

In [ ]:
img = np.array(image)[1].transpose(1, 2, 0)

In [ ]:
plt.imshow(img)

In [ ]:
np.all(img <= 1.0)

In [ ]:
np.all(img >= -1.0)

In [ ]:
img

In [ ]:
batch_sampler = ImageBatchSampler('united', BATCH_SIZE)

In [ ]:
df = batch_sampler.df

In [ ]:
## Group batches
df_n_labels_grouped = df.groupby(by=pd.cut(df['n_labels'], [0, 5, 7, 11, 1000]))
for key1, item1 in df_n_labels_grouped:
    print("\nN LABEL GROUP", key1)
    df1 = df_n_labels_grouped.get_group(key1)
    df_width_grouped = df1.groupby(by=pd.cut(df1['width'], [0, 500, 700, 1000, 10000]))
    for key2, item2 in df_width_grouped:
        print("\tWIDTH GROUP", key2)
        df2 = df_width_grouped.get_group(key2)
        df_height_grouped = df2.groupby(by=pd.cut(df2['height'], [0, 590, 10000]))
        for key3, item3 in df_height_grouped:
            print("\t\tHEIGHT GROUP", key3)
#             print(df_height_grouped.get_group(key3), "\n\n")
            print('\t\t\t', len(df_height_grouped.get_group(key3).index), 'samples')

In [ ]:
df[df['index'].isin(batches[index])]

In [ ]:
len(train_dataset)

In [ ]:
vector.shape